<a href="https://colab.research.google.com/github/vunnampragathi/projects/blob/main/sentimental_analysis_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle

In [2]:
# commands to connect our kaggle account using the json api
! mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
# downloading the data directly from the kaggle usin gthe command API
!kaggle datasets download -d kazanova/sentiment140

 96% 78.0M/80.9M [00:02<00:00, 40.1MB/s]
100% 80.9M/80.9M [00:02<00:00, 33.5MB/s]


In [4]:
# extracting the compressed dataset
from zipfile import ZipFile
dataset='/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print("the dataset is ectracted")



the dataset is ectracted


In [7]:
# Importing the required dependencies

import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
# printing the stop words
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [12]:
# loading the pandas dataframe from the csv file
twitter_data= pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding ='ISO-8859-1')

In [14]:
twitter_data.shape

(1599999, 6)

In [15]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [18]:
# giving the column name
column_names=['target','id','date','flag','user','text']
twitter_data= pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names,encoding ='ISO-8859-1')

In [19]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [20]:
# checking the number of  missing values
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [21]:
# checking the distribution of target column
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [22]:
# converting the labels to 0 and 1. 0 for negative and 1 for positive
twitter_data.replace({'target':{4:1}}, inplace= True)   #inplace= true implies we are changig the data in our original dataset

In [24]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [25]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

#### stemming

In [29]:
port_stem=PorterStemmer()

In [35]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)   #^ means remove all the things except for letters
  stemmed_content=stemmed_content.lower()   # converting all the letters to the lower case
  stemmed_content=stemmed_content.split()   # splitting all the words in the tweet and forming the list
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]   # portstem is to reduce ot the root word. considering each word in port sort and checking whether it is an english stop word or not. if it is not a stopword we will consider that word.
  stemmed_content=' '.join(stemmed_content)   # joins all thw words in the particular tweet

  return stemmed_content


In [ ]:
# now we want to apply this port stem to our data. for that we need to create a new column
twitter_data['stemmed_content']= twitter_data['text'].apply(stemming)

In [32]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,isupsetthathecantupdatehisfacebookbytextingita...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichanidivedmanytimesfortheballmanagedtosave...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,mywholebodyfeelsitchyandlikeitsonfir
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclassnoitsnotbehavingatallimmadwhyam...


In [ ]:
print(twitter_data['stemmed_content'])

In [ ]:
print(twitter_data['target'])

In [ ]:
# seprating thedata and label
X= twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [ ]:
print(X)

In [ ]:
print(Y)

In [ ]:
# splitting the data in to the train and test data
X_train,X_test, Y_train, Y_test= train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=2)

#test size means how much for train data and how much for test data.
# stratify means equal distribution of both classes 0 and 1 in the train and test data.


In [ ]:
print(X.shape,X_train.shape, X_test.shape)

In [ ]:
# converting the textual data in to numeric data. Based on their importance value based on theor repeatance.
vectorizer= TfidfVectorizer()
X_train= vectorizer.fit_transform(X_train)
X_testn= vectorizer.transform(X_test)

In [ ]:
print(X_train)


In [ ]:
print(x_test)


# training the machine learning model

In [ ]:
#logistic regression(classification model)
model= LogisticRegression(max_iter=1000)


In [ ]:
model.fit(X_train, Y_train)

# model evaluation

In [ ]:
# accuracy score on the training data
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(Y_train, X_train_prediction)

In [ ]:
print('accuracy on the training data is :'training_data_accuracy)

In [ ]:
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(Y_test, X_test_prediction)

In [ ]:
print('accuracy on the test data is :'test_data_accuracy)

# saving the trained model

In [ ]:
import pickle

In [ ]:
filename='trained_model.sav'
pickle.dump(model,open(filename,wb))

In [ ]:
#using the saved model fopr future purpose
loaded_model=pickle.load(open('','rb'))

In [ ]:
X_new=X_test[200]
print(Y_test[200])

prediction=loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('negative tweet')

else:
  print('positive tweet')
